In [50]:
# todo:
# view whole distribution
# vary 
#     temperature
#     L, eps
# 3D
# heatmap or contour plot

In [51]:
import time
import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure 
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, Range1d, Slider, Circle
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
from bokeh.palettes import Blues9
from bokeh.plotting import figure, show
from bokeh.sampledata.autompg import autompg as dff

output_notebook()

Loading BokehJS ...

In [52]:
import sys  
sys.path.insert(0, '../../')
from sampling.sampler import Sampler
import jax
import jax.numpy as jnp

nlogp = lambda x: 0.5*jnp.sum(jnp.square(x))
value_grad = jax.value_and_grad(nlogp)

class StandardGaussian():

    def __init__(self, d):
        self.d = d

    def grad_nlogp(self, x):
        """should return nlogp and gradient of nlogp"""
        return value_grad(x)

    def transform(self, x):
        return x[:2]
    #return x

    def prior_draw(self, key):
        """Args: jax random key
           Returns: one random sample from the prior"""

        return jax.random.normal(key, shape = (self.d, ), dtype = 'float64') * 4

targ = StandardGaussian(d = 2)
sampler = Sampler(targ, varEwanted = 5e-4)
init = sampler.Target.prior_draw(key=jax.random.PRNGKey(0))
x, u, l, g, key = sampler.get_initial_conditions(init, jax.random.PRNGKey(0))


/var/folders/v9/977m2z015_b794hqf_nk0q1r0000gn/T/ipykernel_40734/4088906473.py:27: UserWarning: Explicitly requested dtype float64  is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return jax.random.normal(key, shape = (self.d, ), dtype = 'float64') * 4


In [53]:
class bkapp:  
#     df = sea_surface_temperature.copy()
    df = pd.DataFrame(np.array([[1.0],[1.0], [1.0], [1.0]]).T, columns=['x1','x2', 'u1', 'u2'])

    def __init__(self):
        self._theme_json = """
           attrs:
                figure:
                    background_fill_color: "#DDDDDD"
                    outline_line_color: white
                    toolbar_location: above
                    height: 500
                    width: 800
                Grid:
                    grid_line_dash: [6, 4]
                    grid_line_color: white
        """
        
        
        self.xs=[]
        
        self.target = StandardGaussian(d = 2)
        self.sampler = Sampler(targ, varEwanted = 5e-4)
        self.init = sampler.Target.prior_draw(key=jax.random.PRNGKey(0))
        self.x, self.u, self.l, self.g, self.key = sampler.get_initial_conditions(init, jax.random.PRNGKey(0))
        self.L = 10
        self.eps = 1e-1
        self.sigma = jnp.ones(sampler.Target.d)
        self.t = 0
        
        
    def periodically(self):
        self.x, self.u, self.l, self.g, _, self.key, self.t = sampler.dynamics(self.x, self.u, self.g, self.key, self.t, self.L, self.eps, self.sigma)

        self.xs.append(self.x)
        self.xs = self.xs[:100]
        self.source.data = pd.DataFrame(np.array([[self.x[0].item()],[self.x[1].item()], [self.u[0].item()], [self.u[1].item()]]).T, columns=['x1','x2', 'u1', 'u2'])
        
        
    def callbackL(self, attr, old, new):
        self.L = new
    def callbackEps(self, attr, old, new):
        self.eps = new
        
    def kde(self,x, y, N):
        xmin, xmax = x.min(), x.max()
        ymin, ymax = y.min(), y.max()

        X, Y = np.mgrid[xmin:xmax:N*1j, ymin:ymax:N*1j]
        positions = np.vstack([X.ravel(), Y.ravel()])
        values = np.vstack([x, y])
    #     values = np.random.normal(0,1, size=(2,500))
        kernel = gaussian_kde(values)
        Z = np.reshape(kernel(positions).T, X.shape)

        return X, Y, Z

    def __call__(self, doc):
        self.source = ColumnDataSource(data=self.df)
        
        xfig = figure(width=400, height=400)
        xfig.x_range = Range1d(-5, 5)
        xfig.y_range = Range1d(-5, 5)

        
#         xs = np.array(self.xs)
        
#         print(xs.shape)
        
#         x1s, x2s = xs[:,0], xs[:,1]
#         x, y, z = kde(x1s, x2s, 300)

#         co = figure(height=400, width = 400, x_axis_label="", y_axis_label="",
#                    background_fill_color="#fafafa", tools="", toolbar_location=None,
#                    title="Kernel density estimation plot")
#         co.grid.level = "overlay"
#         co.grid.grid_line_color = "black"
#         co.grid.grid_line_alpha = 0.05
        
        xplot = xfig.circle(x='x1', y='x2', source=self.source, size=20, color="navy", alpha=0.5, 
                    fill_alpha=0.6, line_color=None, 
                     hover_fill_color="black", hover_fill_alpha=0.7, hover_line_color=None)

#         palette = Blues9[::-1]
#         levels = np.linspace(np.min(z), np.max(z), 10)
#         co.contour(x, y, z, levels[1:], fill_color=palette, line_color=palette)
        
        ufig = figure(width=400, height=400)
        ufig.x_range = Range1d(-5, 5)
        ufig.y_range = Range1d(-5, 5)
        uplot = ufig.circle(x='u1', y='u2', source=self.source, size=20, color="navy", alpha=0.5)
        

        plot = row([column([xfig]), ufig])
    
        sliderEps = Slider(start=1e-3, end=1e0, value=0.1, step=1e-3, title="Value of epsilon")
        sliderEps.on_change('value', self.callbackEps)
        
        sliderL = Slider(start=1, end=1000, value=10, step=5, title="Value of L")
        sliderL.on_change('value', self.callbackL)
        
        doc.add_periodic_callback(self.periodically, 10)
    
        doc.add_root(column(sliderL, sliderEps, plot))

In [54]:
show(bkapp(), notebook_url="http://localhost:8891") # put the address of the jupyter notebook you are on here!


/var/folders/v9/977m2z015_b794hqf_nk0q1r0000gn/T/ipykernel_40734/4088906473.py:27: UserWarning: Explicitly requested dtype float64  is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return jax.random.normal(key, shape = (self.d, ), dtype = 'float64') * 4
